In [ ]:
import pandas as pd
from google.colab import drive
from os import listdir
import re
import librosa # Pour l'extraction des features et la lecture des fichiers wav
import librosa.display # Pour récupérer les spectrogrammes des audio
import librosa.feature
import os # C'est ce qui va nous permettre d'itérer sur les fichiers de l'environnement de travail
import numpy as np # Pour la normalisation et calculs de moyenne
import cv2
import shutil # gestion des fichiers et dossiers
import pandas as pd # Pour le dataframe
import matplotlib.pyplot as plt # Pour la visualisation
import seaborn as sns # Pour la data visualisation
from pprint import pprint
import tensorflow as tf

# File loading


In [ ]:
#File loading
drive.mount('/content/drive/')
%cp -av "/content/drive/MyDrive/Colab Notebooks/# Project/ProjetMastercamp/df_new.csv" "/content/" 
%cp -av "/content/drive/MyDrive/Colab Notebooks/# Project/ProjetMastercamp/df_3_transform.csv" "/content/"

Mounted at /content/drive/
'/content/drive/MyDrive/Colab Notebooks/# Project/ProjetMastercamp/df_new.csv' -> '/content/df_new.csv'
'/content/drive/MyDrive/Colab Notebooks/# Project/ProjetMastercamp/df_3_transform.csv' -> '/content/df_3_transform.csv'
'/content/drive/MyDrive/Colab Notebooks/# Project/ProjetMastercamp/Data/Data exploration 3/Corona-Hack-Respiratory-Sound-Metadata.csv' -> '/content/Corona-Hack-Respiratory-Sound-Metadata.csv'


Créer les dataframes

In [ ]:
#df des anciens sons
df_new = pd.read_csv('df_new.csv')
df_new = df_new.drop(columns=['Unnamed: 0', 'Unnamed: 0.1'])

#df complet des nouveaux sons(liés aux anciens)
df_3_transform = pd.read_csv('df_3_transform.csv')
df_3_transform = df_3_transform.drop(columns=['Unnamed: 0'])
df_3_transform

,id,AGE,GENDER,Diabetes,Asthma,Smoker,Hypertension,Fever,Cold,Caugh,Muscle_Pain,loss_of_smell,Breathing_Difficulties,Pneumonia,zcr,spectral_c,rolloff,mfcc1,mfcc2,mfcc3,mfcc4,mfcc5,mfcc6,mfcc7,mfcc8,mfcc9,mfcc10,mfcc11,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20,Covid ?
0,/data/train/20200413/vK2bLRNzllXNeyOMudnNSL5cf...,24.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10553.0,2057.615653,3962.563388,-339.446045,33.402565,-2.455190,-5.361792,-1.035550,2.740435,-2.397837,-0.960369,-2.362899,6.329685,4.008686,-2.862794,-2.207973,2.318604,-4.285846,-0.514301,-4.961933,-4.854824,-3.276875,-3.925346,0.0
1,/data/train/20200413/vK2bLRNzllXNeyOMudnNSL5cf...,24.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4557.0,1432.688816,2851.560899,-390.843597,62.604176,20.245140,7.007070,13.764975,11.873083,-0.761298,-3.624951,-6.169991,4.529541,-0.370648,-2.360479,-3.498875,-2.470219,0.573924,3.775793,-5.618447,-6.154309,-1.602978,-6.026536,0.0
2,/data/train/20200413/bjA2KpSxneNskrLBeqi4bqoTD...,72.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30325.0,2422.267305,4544.654297,-474.761749,31.517187,-10.442386,5.461024,-9.016887,-2.251966,-14.545459,-4.542087,-9.414820,-3.064557,3.645913,-4.612294,-5.634139,-2.772233,2.199203,-3.632243,-0.082166,-4.466929,-1.046811,-1.053366,0.0
3,/data/train/20200413/bjA2KpSxneNskrLBeqi4bqoTD...,72.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,21943.0,1516.518654,3048.829784,-653.727844,27.072601,1.040754,8.740533,-7.706318,0.709232,-9.847045,-3.474685,-4.876167,-5.076540,0.154133,-6.040246,-6.274949,-4.002597,-2.759221,-6.111746,-3.171481,-3.547150,-4.328480,-2.914606,0.0
4,/data/train/20200413/FSzobvJqOXf0rI6X05cHqOiU9...,54.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13305.0,2334.792827,3970.367254,-355.057037,58.149475,-71.777336,6.658401,-22.302937,-2.818326,-7.143154,-7.018103,-19.687965,-0.636117,-2.539433,-8.661168,4.544477,-5.736831,-0.733253,-5.783059,2.424303,-3.654979,-0.695806,3.460044,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3183,0,35.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3787.0,1772.306320,3374.159307,-368.879364,68.077347,-31.842756,20.804787,-16.885098,-7.404445,-26.403528,-2.584508,-13.295416,0.436887,-7.446296,-22.604811,1.860285,-4.911074,2.450287,-10.599293,-5.261828,-0.692921,-5.463111,-8.077117,1.0
3184,0,35.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3607.0,1432.937559,2505.809485,-361.175934,68.313095,-48.105083,26.215265,-15.458632,-6.959559,-20.534678,6.545074,-14.389503,-5.681551,-9.024497,-21.722996,8.133307,-6.697014,-2.315064,-8.825060,-6.117575,-4.163162,-6.065608,-5.707136,1.0
3185,0,35.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2428.0,1044.365370,2043.781844,-488.091309,35.424911,-20.825012,16.282202,-10.417974,12.609561,-10.326664,1.328596,-3.122216,-8.382719,-1.694168,-4.429929,-3.683679,-8.873065,0.206908,-6.882071,-5.601245,-2.972606,5.953765,-3.252296,1.0
3186,0,35.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3906.0,1561.828424,2769.825280,-294.453308,80.255058,-56.569901,28.770437,-37.377392,12.972239,-25.391020,-4.120771,-10.994193,-11.015977,-9.469608,-13.095516,5.061044,-9.160578,0.115822,-11.679869,0.923040,-1.656693,5.410047,-8.027106,1.0


#Archives




Renommer les fichiers

In [ ]:
fichiers = listdir('/content/drive/MyDrive//ProjetMastercamp/Data/wav/counting-fast')

#recherche des fichiers
for fichier in fichiers :
    found = re.search('SUB(.+?)_', fichier).group(1)
    if(found != None) :
      fichier_rename = fichier.replace(found, '') #remplacer le nom du fichier
      found = re.search('_(.+?)_', fichier).group(1)
      fichier_rename = fichier_rename.replace("SUB_"+found, '') 
      os.rename("/content/drive/MyDrive//ProjetMastercamp/Data/wav/counting-fast/"+fichier, "/content/drive/MyDrive//ProjetMastercamp/Data/wav/counting-fast/" +fichier_rename)


Data exploration 2 (df des 3000 data) -> Data merge

In [ ]:
df_merge = pd.merge(df, df_categories) #jointure des df
df_merge.set_index(["Unnamed: 0"]) # Définir l’index du DataFrame à l’aide des colonnes existantes
df_merge = df_merge.drop(columns = ['Unnamed: 0','diarrhoea','test_status','covid_status']) #supprimer des colonnes

#fonction qui converti les 0 en 1 (inversement)
print(list(df_merge.columns.values))
def convert_one(x):
  if x == 0: return 1
  else: return 0

#fonction qui converti male en 1 et female en 0
def convert_two(x):
  if x == "male": return 1
  else: return 0

def convert_three(x):
  if x == "covid": return 1
  else: return 0

#Application des fonctions
df_merge['Covid ?'] = df_merge.iloc[:,-1].apply(convert_one) 
df_merge['g'] = df_merge.iloc[:,3].apply(convert_two) 
df_merge = df_merge.fillna(0) # remplace les NaN en 0
df_merge[['smoker', 'cold', 'ht', 'fever', 'diabetes', 'asthma', 'cough', 'loss_of_smell', 'pneumonia']] *= 1

df_merge.to_csv('dfMerge.csv')
print(dfMerge)
print(df_merge.groupby(['Covid ?']).mean() )#moyenne groupée par covid 

Data exploration 2 (Créer un df de pure toux) -> df_new



In [ ]:
#File loading
drive.mount('/content/drive/')
%cp -av "/content/drive/MyDrive/Colab Notebooks/# Project/ProjetMastercamp/df_new.csv" "/content/"
df_new = pd.read_csv('df_new.csv')

# Obtenir un df composé uniquement des données de toux pures
cpt = 0
df_new = pd.DataFrame(columns=df_final.columns)

for i in df_final.values:
  if i[2] == 'cough-shallow':
    df_new.loc[cpt] = i
    cpt +=1
  elif i[2] == 'cough-heavy':
    df_new.loc[cpt] = i
    cpt +=1

#Ajout de donnée ancienne dans le df 
for i in range(0,265):
  df_test['a'] = df_test.iloc[i]['a']=35
for i in range(0,265):
  df_new.loc[i+777] = df_test.iloc[i]

#Filtrage des NaN
df_new = pd.DataFrame(df_new.fillna(0)) # remplace les NaN en 0
print(df_new)
#Suppression des doublons
df_new = df_new.drop_duplicates()
#Save
df_new.to_csv('df_new.csv')
%cp -av "/content/df_new.csv" "/content/drive/MyDrive/Colab Notebooks/# Project/ProjetMastercamp"

data exploration 3 -> df_3

In [ ]:
#File loading
drive.mount('/content/drive/')
%cp -av "/content/drive/MyDrive/Colab Notebooks/# Project/ProjetMastercamp/Data/Data exploration 3/Corona-Hack-Respiratory-Sound-Metadata.csv" "/content/"
df_3 = pd.read_csv('Corona-Hack-Respiratory-Sound-Metadata.csv', encoding = "ISO-8859-1" )

#rendre les covid status positifs différents -> covid simple
df_3.loc[df_3.COVID_STATUS == 'positive_asymp','COVID_STATUS']='covid'
df_3.loc[df_3.COVID_STATUS == 'positive_mild','COVID_STATUS']='covid'
df_3.loc[df_3.COVID_STATUS == 'positive_moderate','COVID_STATUS']='covid'

#genders = {"healthy":0, "positive_mild":1, "recovered_full":0, "positive_mild":1, "positive_moderate":1, "resp_illness_not_identified":0, "no_resp_illness_exposed":0 }
#df_3["COVID_STATUS"] = df_3["COVID_STATUS"].map(genders)

def convert_two(x):
  if x == "M": return 1
  else: return 0

def convert_three(x):
  if x == "covid": return 1
  else: return 0
  
df_3['COVID_STATUS'] = df_3.iloc[:,3].apply(convert_three) 
df_3['GENDER'] = df_3.iloc[:,5].apply(convert_two) 

#Filtrage des NaN
df_3 = pd.DataFrame(df_3.fillna(5))
df_3.drop(df_3[ df_3['cough-heavy'] == 5 ].index , inplace=True)

#Suppression des doublons
df_3 = df_3.drop_duplicates()

#Suppresion des colonnes non nécessaires
df_3 = df_3.drop(columns = ['USER_ID', 'COUNTRY', 'ENGLISH_PROFICIENCY', 'COUNTY_RO_STATE', 'CITY_LOCALITY', 'DATES', 'breathing-deep', 'breathing-shallow', 'counting-fast', 'counting-normal', 'vowel-a', 'vowel-e', 'vowel-o'])


Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
cp: cannot stat '/content/drive/MyDrive/Colab Notebooks/# Project/ProjetMastercamp/Data/Data exploration 3/Corona-Hack-Respiratory-Sound-Metadata.csv': No such file or directory


Ajouter des cas covid -> df_3_transform

In [ ]:
#Nb Covid
mean_df = df_3_transform['Covid ?'].mean()
print(mean_df)

#Transformer les col
df_new.rename(columns = {'a':'AGE', 'g':'GENDER', 'diabetes':'Diabetes', 'asthma':'Asthma','smoker':'Smoker','ht':'Hypertension','fever':'Fever','cold':'Cold','cough':'Caugh','pneumonia':'Pneumonia'}, inplace=True) 
df_3_transform = df_3_transform.drop(columns = ["stage"])
df_new = df_new.drop(columns = ["stage"])

#Insérer à la suite de df_3_transform les df_new['Covid']=1
df = df_new.drop(df_new.loc[df_new['Covid ?'] == 0].index, inplace=True)
df_3_transform = df_3_transform.append(df_new)

#Fillna
df_3_transform = df_3_transform.fillna(0) # remplace les NaN en 0
df_3_transform

#Reordonner l'index

df_3_transform = df_3_transform.reset_index()
df_3_transform = df_3_transform.drop(columns=['index'])

df_3_transform

#Sauvegarder le df_3_transform final

df_3_transform.to_csv('df_3_transform.csv')
shutil.copy("/content/df_3_transform.csv", "/content/drive/MyDrive/ProjetMastercamp")

0.1138129010469436


'/content/drive/MyDrive/ProjetMastercamp/df_3_transform.csv'

# Transformer les fichiers audio en texte -> df

Fonction de transformation

In [ ]:
def audio_pipeline(audio):


  features = []

  # Calcul du ZCR (taux de passage à zéro des audios)

  zcr = librosa.zero_crossings(audio)
  features.append(sum(zcr))

  # Calcul de la moyenne du Spectral centroid

  spectral_centroids = librosa.feature.spectral_centroid(audio)[0]
  features.append(np.mean(spectral_centroids))
  
  # Calcul du spectral rolloff point

  rolloff = librosa.feature.spectral_rolloff(audio)
  features.append(np.mean(rolloff))

  # Calcul des moyennes des MFCC (Mel-frequency cepstral coefficients)

  mfcc = librosa.feature.mfcc(audio)

  for x in mfcc:
    features.append(np.mean(x))

  return features


Créer les colonnes de df_3_transform

In [ ]:
column_names = ['id', 'AGE', 'GENDER', 'Diabetes', 'Asthma', 'Smoker', 'Hypertension', 'Fever', 'Cold', 'Caugh', 'Muscle_Pain', 'loss_of_smell',
 'Breathing_Difficulties', 'Pneumonia','zcr', 'spectral_c', 'rolloff', 'mfcc1', 'mfcc2', 'mfcc3',
                'mfcc4', 'mfcc5', 'mfcc6', 'mfcc7', 'mfcc8', 'mfcc9',
                'mfcc10', 'mfcc11', 'mfcc12', 'mfcc13', 'mfcc14', 'mfcc15',
                'mfcc16', 'mfcc17', 'mfcc18', 'mfcc19', 'mfcc20','Covid ?']

#attribuer les colonnes dans le dataframe
df_3_transform = pd.DataFrame(columns = column_names)


Itérer la transformation à tous les fichiers wav -> df_3_transform

In [ ]:
compt = 0
info_stat = 0
cpt = 0

# Transformer le fichier wav -> dataframe et l'insérer dans df 
for i in range(0,len(df_3)) :
  print(info_stat)
  info_stat += 1
  if i==621:
    i+=1
  elif i==1148:
    i+=1

  try :

    extract = librosa.load("/content/drive/MyDrive/Colab Notebooks/# Project/ProjetMastercamp/Data/Data exploration 3/"+ df_3.loc[i]['cough-heavy'])[0]
    df_3_transform.loc[cpt] = [df_3.iloc[i]['cough-heavy']] + [df_3.iloc[i]['AGE']] + [df_3.iloc[i]['GENDER']]+ [df_3.iloc[i]['Diabetes']]+ [df_3.iloc[i]['Asthma']]+ [df_3.iloc[i]['Smoker']]+ [df_3.iloc[i]['Hypertension']]+ [df_3.iloc[i]['Fever']]+ [df_3.iloc[i]['Cold']]+ [df_3.iloc[i]['Caugh']] + [df_3.iloc[i]['Muscle_Pain']]+ [df_3.iloc[i]['loss_of_smell']] + [df_3.iloc[i]['Breathing_Difficulties']] + [df_3.iloc[i]['Pneumonia']]+ audio_pipeline(extract) + [df_3.iloc[i]['COVID_STATUS']]
    cpt+=1

    extract = librosa.load("/content/drive/MyDrive/Colab Notebooks/# Project/ProjetMastercamp/Data/Data exploration 3/"+ df_3.loc[i]['cough-shallow'])[0]
    df_3_transform.loc[cpt] = [df_3.iloc[i]['cough-shallow']] + [df_3.iloc[i]['AGE']] + [df_3.iloc[i]['GENDER']]+ [df_3.iloc[i]['Diabetes']]+ [df_3.iloc[i]['Asthma']]+ [df_3.iloc[i]['Smoker']]+ [df_3.iloc[i]['Hypertension']]+ [df_3.iloc[i]['Fever']]+ [df_3.iloc[i]['Cold']]+ [df_3.iloc[i]['Caugh']] + [df_3.iloc[i]['Muscle_Pain']]+ [df_3.iloc[i]['loss_of_smell']] + [df_3.iloc[i]['Breathing_Difficulties']] + [df_3.iloc[i]['Pneumonia']]+ audio_pipeline(extract) + [df_3.iloc[i]['COVID_STATUS']]
    cpt+=1

  # Regarder la taille du file et s'il existe
  except FileNotFoundError :
    print('File not found next step')
    compt += 1
  except ValueError :
    print('File is so small to convert')
    compt += 1

df_3_transform
df_3_transform.to_csv('df_3_transform.csv')
shutil.copy("/content/df_3_transform.csv", "/content/drive/MyDrive/Colab Notebooks/# Project/ProjetMastercamp")

# Transformer les fichiers audio en image -> df

Transformer l'audio en image (spectrogramme) et placer les images dans les dossiers correspondants

In [ ]:
def image_pipeline(name_file,n,covid):
#Transformer son en image
  y, sr = librosa.load(name_file)
  S = librosa.feature.melspectrogram(y, sr, n_mels=128, fmax=8000) 
  librosa.display.specshow(librosa.power_to_db(S, ref=np.max), fmax=8000) 

#Save image
  new_name = 'image_'+str(n)+'.png'
  plt.savefig(new_name)
  img = cv2.imread(new_name)
  print(img.shape)

#Placer image
  if covid == 0:
   shutil.copy("/content/"+new_name, "/content/drive/MyDrive/Colab Notebooks/# Project/ProjetMastercamp/Data/image/pasCovid")
  else:
    shutil.copy("/content/"+new_name, "/content/drive/MyDrive/Colab Notebooks/# Project/ProjetMastercamp/Data/image/Covid")
  os.remove("/content/"+new_name)


Itérer la transformation à tous les wav 

In [ ]:
compt = 0
info_stat = 0
for i in range(0,len(df_final)) :
  print(info_stat)
  info_stat += 1 
  n = i - compt

  try :
    # Transformer le file en image
    extract = "/content/drive/MyDrive/Colab Notebooks/# Project/ProjetMastercamp/Data/wav/"+ df_final.loc[i]['stage'] + "/" + df_final.loc[i]['stage'] + "_" + df_final.loc[i]['id'] + ".wav" #[0] ?
    image_pipeline(extract,n,df_final.loc[i]['Covid ?'])
    
  except FileNotFoundError :
    print('File not found next step')
    compt += 1
  except ValueError :
    print('File is so small to convert')
    compt += 1


Déterminer la taille de chacun des dataset

In [ ]:
#dossier image avec toutes les data
print(len(listdir('/content/drive/MyDrive/Colab Notebooks/# Project/ProjetMastercamp/Data/image/Covid')))
print(len(listdir('/content/drive/MyDrive/Colab Notebooks/# Project/ProjetMastercamp/Data/image/pasCovid')))
print("\n")

#dossier image_toux avec seulement les toux
print(len(listdir('/content/drive/MyDrive/Colab Notebooks/# Project/ProjetMastercamp/Data/image_toux/Covid')))
print(len(listdir('/content/drive/MyDrive/Colab Notebooks/# Project/ProjetMastercamp/Data/image_toux/Pas_Covid')))

577
2751


178
761


# Data Test

Transformer un son test en texte

In [ ]:
%cp -av "/content/drive/MyDrive/Colab Notebooks/# Project/ProjetMastercamp/test_positif.wav" "/content/"
#%cp -av "/content/drive/MyDrive/ProjetMastercamp/test_positif.wav" "/content/"

def transformation(id, AGE, GENDER, Diabetes, Asthma, Smoker, Hypertension, Fever, Cold, Caugh, Muscle_Pain, loss_of_smell, Breathing_Difficulties, Pneumonia, name_file):
  #partie non-supervisée
  column_names1 = ['zcr', 'spectral_c', 'rolloff', 'mfcc1', 'mfcc2', 'mfcc3',
                  'mfcc4', 'mfcc5', 'mfcc6', 'mfcc7', 'mfcc8', 'mfcc9',
                  'mfcc10', 'mfcc11', 'mfcc12', 'mfcc13', 'mfcc14', 'mfcc15',
                  'mfcc16', 'mfcc17', 'mfcc18', 'mfcc19', 'mfcc20']
  df_user = pd.DataFrame(columns = column_names1)

  #partie supervisée
  column_names = ['id', 'AGE', 'GENDER', 'Diabetes', 'Asthma', 'Smoker', 'Hypertension', 'Fever', 'Cold', 'Caugh', 'Muscle_Pain', 'loss_of_smell','Breathing_Difficulties', 'Pneumonia']
  ar = np.array([id, AGE, GENDER, Diabetes, Asthma, Smoker, Hypertension, Fever, Cold, Caugh, Muscle_Pain, loss_of_smell, Breathing_Difficulties, Pneumonia])
  df_user2 = pd.DataFrame(ar, column_names).T

  #transformation partie non-supervisée
  extract = librosa.load(name_file)[0]
  df_user.loc[0] = audio_pipeline(extract) 

  #jointure
  df_test = df_user2.join(df_user)

  #save et load
  df_test.to_csv('test.csv')
  shutil.copy("/content/test.csv", "/content/drive/MyDrive/Colab Notebooks/# Project/ProjetMastercamp")
  os.remove("/content/test.csv")

#test
transformation(20,1,0,1,0,0,0,1,0,1,0,1,0,0,'test_positif.wav')

'/content/drive/MyDrive/Colab Notebooks/# Project/ProjetMastercamp/test_positif.wav' -> '/content/test_positif.wav'


Prédiction

Transformer un son test en image

In [ ]:
extract = "/content/drive/MyDrive/ProjetMastercamp/Avenue-de-la-République-2.wav" 

def image_pipeline_test(extract,n):

  new_name = 'image_'+str(n)+'.png'
  
  y, sr = librosa.load(extract)
  S = librosa.feature.melspectrogram(y, sr, n_mels=128, fmax=8000) 
  librosa.display.specshow(librosa.power_to_db(S, ref=np.max), fmax=8000) 

  plt.savefig(new_name)
  img = cv2.imread(new_name)
  
  shutil.copy(#path)
  os.remove("/content/"+new_name)


Prédiction